In [1]:
import numpy as np
from astropy.io import fits
%matplotlib inline
from matplotlib import pyplot as plt
from astropy.visualization import hist
import ccdproc as ccdp
from matplotlib import rc
import os
import time
import glob

from astropy import units as u
from astropy.modeling import fitting
from astropy.modeling.models import Polynomial1D,Chebyshev1D,Legendre1D,Hermite1D

from astropy.nddata import CCDData
from datetime import datetime

phot_tutorial_dir = '/Users/polinapetrov/F21/ASTR8060/phot_tutorial/notebooks'
plt.style.use(phot_tutorial_dir+'/guide.mplstyle')
rc('font', size=12)
rc('axes', grid=True)

In [2]:
data_dir = '/Users/polinapetrov/F21/ASTR8060/Imaging/' #raw data
reduced1 = '/Users/polinapetrov/F21/ASTR8060/reduced/' #reduced directory used originally
reduced_dir = '/Users/polinapetrov/F21/ASTR8060/reduced_2/' #new reduced directory, starting over

In [50]:
#adding biassec and trimsec to headers of ALL frames
os.chdir(reduced_dir)
all_files = [file for file in glob.glob('*.fits')]
all_files = sorted(all_files)

hdus = [fits.open(file) for file in all_files]
hdrs = [hdu[0].header for hdu in hdus]
data = [hdu[0].data for hdu in hdus]

for i in range(np.size(hdus)):
    hdrs[i]['BIASSEC'] = '[2102:2200,1:2048]'
    hdrs[i]['TRIMSEC'] = '[54:2101,1:2048]'
    hdus[i].writeto(all_files[i], overwrite=True)
    
del hdus
del hdrs

In [51]:
#adding filter and imagetyp to headers for bias frames
nfiles = 19
nlow = 93
bias_files = [reduced_dir+'a'+str(i).zfill(3)+'.fits' for i in np.arange(nfiles)+nlow]
hdus = [fits.open(file) for file in bias_files]
hdrs = [hdu[0].header for hdu in hdus]
biases = [hdu[0].data for hdu in hdus]

for i in range(np.size(hdus)):
    hdrs[i]['FILTER'] = 'none'
    hdrs[i]['IMAGETYP'] = 'bias'
    hdus[i].writeto(bias_files[i], overwrite=True)
del hdus
del hdrs

In [99]:
#opening bias frames
imgs = ccdp.ImageFileCollection(reduced_dir,glob_include='*.fits')
bias_files = imgs.files_filtered(imagetyp='bias',include_path=True)
biases = [CCDData.read(file,unit='adu') for file in bias_files]

In [100]:
#fitting and subtracting the overscan region from all frames
#trimming all frames so that only the 2048x2048 data section remains
model = Polynomial1D(3)
imgs = ccdp.ImageFileCollection(reduced_dir,glob_include='*.fits',glob_exclude='*ot.fits')
for ccd, file_name in imgs.ccds(ccd_kwargs={'unit': 'adu'},return_fname=True):
    try: 
        ccd_sav = ccd
        ccd = ccdp.subtract_overscan(ccd, fits_section=ccd.header['biassec'], median=False, model=model)
        ccd = ccdp.trim_image(ccd,fits_section=ccd.header['trimsec'])
        outfile = file_name.strip('.fits')+'ot.fits'
        ccd.write(reduced_dir+outfile, overwrite=True)
    except:
        print("Had trouble with file \"{}\". Skipping...".format(file_name))

In [101]:
#creating master bias, excluding first frame
#(the mean level in the first bias frame clearly stands out compared to the other bias frames)
imgs = ccdp.ImageFileCollection(reduced_dir,glob_include='*ot.fits')
biasFiles = imgs.files_filtered(imagetyp='bias',include_path=True)
bias_restricted = biasFiles[1:]
bias_mn_master = ccdp.combine(bias_restricted,method='average',unit='adu',sigma_clip=False)

INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


...

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


In [102]:
#subtract master bias from all frames
imgs_ot = ccdp.ImageFileCollection(reduced_dir,glob_include='*ot.fits')
for ccd, file_name in imgs_ot.ccds(ccd_kwargs={'unit': 'adu'},return_fname=True):
    try: 
        ccd_sav = ccd
        ccd = ccdp.subtract_bias(ccd, bias_mn_master)
        outfile = file_name.strip('ot.fits')+'otz.fits'
        ccd.write(reduced_dir+outfile, overwrite=True)
    except:
        print("Had trouble with file \"{}\". Skipping...".format(file_name))

INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


...

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


# Other things to do before doing HW 5

In [115]:
# get V-band filenames
nfiles = 7
nlow = 43

V_files = [reduced_dir+'a'+str(i).zfill(3)+'otz.fits' for i in np.arange(nfiles)+nlow]
hdus = [fits.open(file) for file in V_files]
hdrs = [hdu[0].header for hdu in hdus]
Vs = [hdu[0].data for hdu in hdus]

for i in range(len(hdus)):
    hdrs[i]['FILTER'] = 'V'
    hdrs[i]['IMAGETYP'] = 'skyflat'
    hdus[i].writeto(V_files[i], overwrite=True) 

In [116]:
# get U-band filenames
nfiles = 8
nlow = 21
U_files = [reduced_dir+'a'+str(i).zfill(3)+'otz.fits' for i in np.arange(nfiles)+nlow]
hdus = [fits.open(file) for file in U_files]
hdrs = [hdu[0].header for hdu in hdus]
for i in range(len(hdus)):
    hdrs[i]['FILTER'] = 'U'
    hdrs[i]['IMAGETYP'] = 'skyflat'
    hdus[i].writeto(U_files[i], overwrite=True) 

In [117]:
# get B-band filenames
nfiles = 14
nlow = 29
B_files = [reduced_dir+'a'+str(i).zfill(3)+'otz.fits' for i in np.arange(nfiles)+nlow]
hdus = [fits.open(file) for file in B_files]
hdrs = [hdu[0].header for hdu in hdus]
for i in range(len(hdus)):
    hdrs[i]['FILTER'] = 'B'
    hdrs[i]['IMAGETYP'] = 'skyflat'
    hdus[i].writeto(B_files[i], overwrite=True)

In [118]:
# get R-band filenames
nfiles = 6
nlow = 50
R_files = [reduced_dir+'a'+str(i).zfill(3)+'otz.fits' for i in np.arange(nfiles)+nlow]
hdus = [fits.open(file) for file in R_files]
hdrs = [hdu[0].header for hdu in hdus]
for i in range(len(hdus)):
    hdrs[i]['FILTER'] = 'R'
    hdrs[i]['IMAGETYP'] = 'skyflat'
    hdus[i].writeto(R_files[i], overwrite=True)

In [119]:
# get I-band filenames
nfiles = 7
nlow = 56
I_files = [reduced_dir+'a'+str(i).zfill(3)+'otz.fits' for i in np.arange(nfiles)+nlow]
hdus = [fits.open(file) for file in I_files]
hdrs = [hdu[0].header for hdu in hdus]
for i in range(len(hdus)):
    hdrs[i]['FILTER'] = 'I'
    hdrs[i]['IMAGETYP'] = 'skyflat'
    hdus[i].writeto(I_files[i], overwrite=True)

In [3]:
# get H-alpha filenames
nfiles = 16
nlow = 5
Ha_files = [reduced_dir+'a'+str(i).zfill(3)+'otz.fits' for i in np.arange(nfiles)+nlow]
hdus = [fits.open(file) for file in Ha_files]
hdrs = [hdu[0].header for hdu in hdus]
for i in range(len(hdus)):
    hdrs[i]['FILTER'] = 'Halpha'
    hdrs[i]['IMAGETYP'] = 'skyflat'
    hdus[i].writeto(Ha_files[i], overwrite=True)